In [1]:
def waitrobot(robot):
    """busy wait for robot completion"""
    while not robot.GetController().IsDone():
        time.sleep(0.01)

In [2]:
import time
import openravepy
import numpy as np
env = openravepy.Environment()
env.SetViewer('qtcoin')
env.Load('data/pr2test2.env.xml')
robot = env.GetRobots()[0]

#manip = robot.SetActiveManipulator('leftarm_torso') # set the manipulator to leftarm + torso
manip = robot.SetActiveManipulator('leftarm') # set the manipulator to leftarm
ikmodel = openravepy.databases.inversekinematics.InverseKinematicsModel(robot,iktype=openravepy.IkParameterization.Type.Transform6D)
if not ikmodel.load():
    ikmodel.autogenerate()
basemanip = openravepy.interfaces.BaseManipulation(robot)
taskprob = openravepy.interfaces.TaskManipulation(robot)

target=env.GetKinBody('TibitsBox1')
with env:
    jointnames = ['l_shoulder_lift_joint','l_elbow_flex_joint','l_wrist_flex_joint','r_shoulder_lift_joint','r_elbow_flex_joint','r_wrist_flex_joint']
    robot.SetActiveDOFs([robot.GetJoint(name).GetDOFIndex() for name in jointnames])
    basemanip.MoveActiveJoints(goal=[1.29023451,-2.32099996,-0.69800004,1.27843491,-2.32100002,-0.69799996])
robot.WaitForController(0)

No handlers could be found for logger "openravepy.ikfast"


KeyboardInterrupt: 

In [ ]:
print 'move robot base to target'
teleport = False
with env:
    if teleport:
        th = 0
        z = robot.GetTransform()[2,3]
        th = 0        
        x, y = 2.8, -1.3
        T = openravepy.matrixFromAxisAngle([0,0,th])
        T[:, -1] = [x, y, z, 1]
        robot.SetTransform(T)
    else:
        robot.SetActiveDOFs([],openravepy.DOFAffine.X|openravepy.DOFAffine.Y|openravepy.DOFAffine.RotationAxis,[0,0,1])
        basemanip.MoveActiveJoints(goal=[2.8,-1.3,0],maxiter=5000,steplength=0.15,maxtries=2)

In [ ]:
robot.WaitForController(0)
taskprob.ReleaseFingers()

In [ ]:
print 'move the arm to the target'
Tgoal = np.array([[0,-1,0,3.5],[-1,0,0,-1.3],[0,0,-1,0.842],[0,0,0,1]])
res = basemanip.MoveToHandPosition(matrices=[Tgoal],seedik=16)
robot.WaitForController(0)

In [ ]:
taskprob.CloseFingers()
robot.WaitForController(0)

In [ ]:
print 'move the arm with the target back to the initial position'
with env:
    robot.Grab(target)
    basemanip.MoveManipulator(goal=[0, 0, 1.29023451, 0, -2.32099996, 0, -0.69800004, 0])
robot.WaitForController(0)

In [ ]:
print 'move the robot to another location'
with env:
    robot.SetActiveDOFs([],openravepy.DOFAffine.X|openravepy.DOFAffine.Y|openravepy.DOFAffine.RotationAxis,[0,0,1])
    localgoal = [0,2.4,0]
    T = robot.GetTransform()
    goal = np.dot(T[0:3,0:3],localgoal) + T[0:3,3]
    with robot:
        robot.SetActiveDOFValues(goal)
        incollision = env.CheckCollision(robot)
        if incollision:
            print 'goal in collision!!'

In [ ]:
basemanip.MoveActiveJoints(goal=goal,maxiter=5000,steplength=0.15,maxtries=2)
robot.WaitForController(0)

In [ ]:
print 'move the arm to the designated position on another table to place the target down'
Tgoal = np.array([[0,-1,0,3.5],[-1,0,0,1.5],[0,0,-1,0.855],[0,0,0,1]])
res = basemanip.MoveToHandPosition(matrices=[Tgoal],seedik=16)
robot.WaitForController(0)

In [9]:
taskprob.ReleaseFingers(target=target)
robot.WaitForController(0)

True

In [10]:
print 'move manipulator to initial position'
basemanip.MoveManipulator(goal=[0, 0, 1.29023451, 0, -2.32099996, 0, -0.69800004, 0])
robot.WaitForController(0)

move manipulator to initial position


True

In [11]:
print 'close fingers until collision'
taskprob.CloseFingers()
robot.WaitForController(0)

close fingers until collision


True